### **Tax_Getter**

The goal of this web-scraping application is to automatically compile
a list of all tax rates by zip code and generate a CSV file that
can be used for our business

In [1]:
import time # So that we don't bombard the site with requests and annoy them
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "http://www.sale-tax.com/" # Website to collect State / Abbreviations list from

response = requests.get(url) # Collecting HTML data from website

site = BeautifulSoup(response.content, "html.parser") # Parsing HTML for string

line_list = site.text.split('\n') # Splitting by line

states = [line.split(')')[0] for line in line_list 
          if 'sales tax rates' in line.lower()]      # Collecting a list of all states

all_states = states[2:-6]

In [3]:
# Turns the states into a dictionary that
# relates the two-letter state code
# to the full state name (used in 
# get_tax_data())

states_dict = {}
for state in all_states:
    state_info = state.split('(')
    state_name = state_info[0][:-1]
    state_code = state_info[1]
    states_dict[state_name] = state_code

In [4]:
# Returns the list of state strings!

states = list(states_dict.keys())

In [5]:
# Retrieves all the city taxes for each state

def get_tax_data(state):
    
    url = f"http://www.sale-tax.com/{state.replace(' ', '')}_all"
    
    response = requests.get(url)
    
    site = BeautifulSoup(response.content, "html.parser")
    
    county_tax_data = filter(None, site.text.split('\n'))
    
    # Dictionary of lists that will be converted to
    # a dataframe
    
    counties = {'State_Abbrev': [],
                'City': [],
                'State_Rate': [],
                'Combined_Rate': []}
    
    is_county = False
    for row, info in enumerate(county_tax_data):
                
        if f'The state sales tax rate in {state} is ' in info:
            # The state sales tax at the top of the page
            # as a decimal
            state_tax = float(info.split(' ')[-1].split('%')[0])/100
        
        # Flag the beginning of data collection for
        # each county
        if 'Show: Popular | Counties | All |' in info:
            is_county = False
            
        if is_county:
            county_info = info.split(' Sales Tax Rate')
            counties['City'] += [county_info[0].split(', ')[0]]
            counties['State_Abbrev'] += [county_info[0].split(', ')[1]]
            counties['State_Rate'] += [state_tax]
            counties['Combined_Rate'] += [float(county_info[1].replace("%", ""))/100]
        
        if 'CityTotal Sales Tax Rate' in info:
            is_county = True
    
    df = pd.DataFrame.from_dict(counties)
    df['City/CountyRates'] = df['Combined_Rate'] - df['State_Rate']
    
    return df

In [6]:
# Retrieves all the zipcodes associated with a state

def get_zips(state):
    
    url_zips = f"https://www.zip-codes.com/state/{states_dict[state]}.asp"
    response_zips = requests.get(url_zips)
    zips_site = BeautifulSoup(response_zips.content, "html.parser")
    
    site_data = list(filter(None, zips_site.find_all("a", href=True))) # collect site data
    
    zipcodes = {"Zipcode": [],
                "City": [],
                "County": [],
                "State_Abbrev": []} # Necessary when merging dataframes!
    
    zipcode_info = False
    
    data = []
    
    for line in site_data:
        
        line = line.string
        
        if line is not None:
        
            # Flag the end of our data collection
            # for zipcodes
            
            if line == "More Census Information":
                zipcode_info = False
            
            elif zipcode_info:
                if "ZIP Code" in line:
                    data.append(states_dict[state]) # Add the state abbrev
                    line = line.split(' ')[-1]
                data.append(line)
        
            # Flag the beginning of our data 
            # collection for zipcodes
            
            elif line == "More Details":
                zipcode_info = True
    
    # Generate a dictionary to turn into dataframe
    
    info_type = ["State_Abbrev", "Zipcode", "City", "County"]
    for index, info in enumerate(data):
        info_index = index % 4
        zipcodes[info_type[info_index]] += [info]
    
    return pd.DataFrame.from_dict(zipcodes)

In [7]:
# Executes both functions for each state.
# Enumerated so that the first state
# analyzed serves as the start of the 
# dataframe

for state_num, state in enumerate(states):
    
    if state_num == 0:
        df = get_tax_data(state)
        df_zips = get_zips(state)
    else:
        df = df.append(get_tax_data(state), ignore_index=True)
        df_zips = df_zips.append(get_zips(state), ignore_index=True)
        
    time.sleep(0.25)
    
    print(f"Completed {state}... {len(states)-state_num-1} states remaining.")

Completed Alabama... 49 states remaining.
Completed Alaska... 48 states remaining.
Completed Arizona... 47 states remaining.
Completed Arkansas... 46 states remaining.
Completed California... 45 states remaining.
Completed Colorado... 44 states remaining.
Completed Connecticut... 43 states remaining.
Completed Delaware... 42 states remaining.
Completed Florida... 41 states remaining.
Completed Georgia... 40 states remaining.
Completed Hawaii... 39 states remaining.
Completed Idaho... 38 states remaining.
Completed Illinois... 37 states remaining.
Completed Indiana... 36 states remaining.
Completed Iowa... 35 states remaining.
Completed Kansas... 34 states remaining.
Completed Kentucky... 33 states remaining.
Completed Louisiana... 32 states remaining.
Completed Maine... 31 states remaining.
Completed Maryland... 30 states remaining.
Completed Massachusetts... 29 states remaining.
Completed Michigan... 28 states remaining.
Completed Minnesota... 27 states remaining.
Completed Mississipp

In [11]:
# Combines the two dataframes on 'city' so that
# each zipcode is associated with a given
# sales tax rate

sales_tax_data = (pd.merge(df_zips, df, on=["State_Abbrev", "City"])
                    .drop_duplicates())

# Re-arranging columns
cols = ['Zipcode', 'State_Abbrev', 'City', 'County', 
        'State_Rate', 'City/CountyRates', 'Combined_Rate']

sales_tax_data = sales_tax_data[cols]

In [9]:
# Save this dataframe as a CSV

path = "C:/Users/William/Documents/_UL/TaxGetter/Data/TaxData"
sales_tax_data.to_csv(path + ".csv", index=False)
#sales_tax_data.to_excel(path + ".xlsx", index=False) # Could be useful, takes much longer than CSV though!

In [10]:
# Accomplished with reference to
# http://www.sale-tax.com/
# https://www.zip-codes.com/
# https://www.geeksforgeeks.org/how-to-convert-dictionary-to-pandas-dataframe/
# https://www.geeksforgeeks.org/python-pandas-dataframe-append/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all 
# https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns (How to re-order dataframe columns)